In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics
from sklearn.utils import shuffle
from numpy import asarray
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping

import sys
sys.path.append('../')
from Utilities.model_visualization import model_to_png

print('import done')

C:\Users\Javapocalypse\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


import done


In [33]:
df = pd.read_excel('tweets.xlsx')


In [34]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 6131 unique tokens.


In [35]:
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (3616, 250)


In [36]:
Y = pd.get_dummies(df['label']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (3616, 4)


In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3254, 250) (3254, 4)
(362, 250) (362, 4)


In [41]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 404       
Total params: 5,080,804
Trainable params: 5,080,804
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
epochs = 100
batch_size = 100

history = model.fit(
    X_train, 
    Y_train, 
    epochs=epochs, 
    batch_size=batch_size,
    validation_split=0.1,
    verbose=1,
    callbacks=[
        EarlyStopping(monitor='val_acc', patience=4, min_delta=0.0001)
    ]
)

Train on 2928 samples, validate on 326 samples
Epoch 1/100
2928/2928 [==============================] - 11s 4ms/step - loss: 0.1863 - acc: 0.9529 - val_loss: 1.1292 - val_acc: 0.6595
Epoch 2/100
2928/2928 [==============================] - 11s 4ms/step - loss: 0.1096 - acc: 0.9747 - val_loss: 1.2472 - val_acc: 0.6595
Epoch 3/100
2928/2928 [==============================] - 11s 4ms/step - loss: 0.0767 - acc: 0.9836 - val_loss: 1.2571 - val_acc: 0.6626
Epoch 4/100
2928/2928 [==============================] - 11s 4ms/step - loss: 0.0560 - acc: 0.9850 - val_loss: 1.2508 - val_acc: 0.6718
Epoch 5/100
2928/2928 [==============================] - 11s 4ms/step - loss: 0.0395 - acc: 0.9908 - val_loss: 1.3706 - val_acc: 0.6779
Epoch 6/100
2928/2928 [==============================] - 11s 4ms/step - loss: 0.0258 - acc: 0.9952 - val_loss: 1.4308 - val_acc: 0.6748
Epoch 7/100
2928/2928 [==============================] - 11s 4ms/step - loss: 0.0189 - acc: 0.9959 - val_loss: 1.5117 - val_acc: 0.6810
E

In [31]:
model.load_weights('lstm_text.h5')


In [40]:
new_complaint = ['RT #Earthquake in #kuwait Everyone is standing outside']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['dont know','little','mild','severe']
print(pred, labels[np.argmax(pred)])

[[0.96619326 0.01906555 0.01218033 0.00256095]] dont know


In [22]:
def predict(text):
    df = pd.read_excel('tweets.xlsx')
    
    # The maximum number of words to be used. (most frequent)
    MAX_NB_WORDS = 50000
    # Max number of words in each complaint.
    MAX_SEQUENCE_LENGTH = 250
    # This is fixed.
    EMBEDDING_DIM = 100

    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
    tokenizer.fit_on_texts(df['text'].values)
    word_index = tokenizer.word_index
    
    X = tokenizer.texts_to_sequences(df['text'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    
    Y = pd.get_dummies(df['label']).values

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)

    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(4, activation='softmax'))
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.load_weights('lstm_text.h5')

    new_complaint = ['RT #Earthquake in #kuwait Everyone is standing outside']
    seq = tokenizer.texts_to_sequences(new_complaint)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded)
    labels = ['Don\'t know or can\'t say','Little Damage','Mild Damage','Severe Damage']
    high_conf = pred[0][3]
    moderate_conf = pred[0][2]
    low_conf = pred[0][1]
    print(text, 'Classified as: '+labels[np.argmax(pred)])
    return high_conf, low_conf, moderate_conf

In [23]:
predict('RT #Earthquake in #kuwait Everyone is standing outside')

RT #Earthquake in #kuwait Everyone is standing outside Classified as: Little Damage


(0.027757093, 0.82317704, 0.121885866)